In [ ]:
!pip install httpx

In [ ]:
import hmac
import hashlib
from urllib.parse import urlencode

import httpx

API_BASE = "https://api.sanatdhir.com"
AGENT_ID = "https://agent.sanatdhir.com"

# IMPORTANT:
#   Replace this with the exact same value you set in the Lambda
#   environment variable AGENT_SHARED_SECRET.
AGENT_SHARED_SECRET = "Ztq_9sB2kP_ExampleSecret123"


def build_canonical(method: str, path: str, params: dict) -> (str, str):
    """
    Build the canonical string:
        METHOD \n PATH \n rawQueryString \n Signature-Agent

    Returns (canonical_string, raw_query_string).
    """
    method = method.upper()
    # Build the raw query string once so we sign EXACTLY what we send
    query_str = urlencode(params or {}, doseq=True)
    canonical = f"{method}\n{path}\n{query_str}\n{AGENT_ID}"
    return canonical, query_str


def sign_request(method: str, path: str, params: dict) -> (str, str):
    """
    Return (signature_hex, raw_query_string) for this request.
    """
    canonical, query_str = build_canonical(method, path, params)
    sig_hex = hmac.new(
        AGENT_SHARED_SECRET.encode("utf-8"),
        canonical.encode("utf-8"),
        hashlib.sha256,
    ).hexdigest()
    return sig_hex, query_str


def call_flights_as_agent(origin="SFO", destination="JFK", date="2025-12-01"):
    """
    Call the HAP Travel API as an authenticated agent using HAP-Sig (demo HMAC).
    """
    path = "/api/flights/search"
    params = {
        "from": origin,
        "to": destination,
        "date": date,
    }

    sig_hex, query_str = sign_request("GET", path, params)

    # Build URL with the same query string we signed
    url = f"{API_BASE}{path}?{query_str}"
    headers = {
        "Sec-Client-Class": "agent",
        "Signature-Agent": AGENT_ID,
        "Authorization": f"HAP-Sig {sig_hex}",
    }

    with httpx.Client() as client:
        resp = client.get(url, headers=headers)

    print("Request URL:", url)
    print("Status:", resp.status_code)
    print("Response headers:", dict(resp.headers))
    print("Body:", resp.text)


# Example usage:
call_flights_as_agent("SFO", "JFK", "2025-12-01")
